# Topic Modeling
- 비지도학습

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출. 
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

# 위에서 cats 변수로 기재된 category만 추출. featch_20newsgroups( )의 categories에 cats 입력
news_df= fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), 
                            categories=cats, random_state=0)

In [5]:
# 벡터화
# LDA 는 Count기반의 Vectorizer만 적용합니다.  
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


In [6]:
# 8개로 분리(하이퍼 파라미터 지정)
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

##### 각 토픽 모델링 주제별 단어들의 연관도 확인
- LDA 객체의 components_ 속성은 주제별로 개별 단어들의 연관도 정규화 숫자가 들어있음
- shape는 주제 개수 X 피처 단어 개수
- components_에 들어 있는 숫자값은 각 주제별로 단어가 나타난 횟수를 정규화 하여 나타냄
- 숫자가 클 수록 토픽에서 단어가 차지하는 비중이 높음

In [7]:
lda.components_.shape

(8, 1000)

In [10]:
lda.components_[0, :10]

array([ 36.09920176, 135.62679793,  21.57518668,  16.67973737,
        27.91164531,  51.08874696,  27.15781184, 563.62985361,
       151.25501234, 250.00265828])

### 단어별 토픽 분류 확인

In [19]:
def display_topic_words(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)
        
        # components_ array에서 가장 값이 큰순으로 정렬했을때, 그 값의 array index를 반환
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:no_top_words]
        
        # top_indexes 대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' + '.join(str(feature_names[i]) + ' * ' + str(round(topic[i],1)) for i in top_indexes)
        print(feature_concat)

In [17]:
# 단어들을 sort해서 보여줌
len(count_vect.get_feature_names())

1000

In [21]:
# CountVectorizer객체내의 전체 word들의 명칭을 get_features_name를 초해 추출
feature_names = count_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topic_words(lda, feature_names, 15)

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출.

Topic # 0
year * 703.2 + 10 * 563.6 + game * 476.3 + medical * 413.2 + health * 377.4 + team * 346.8 + 12 * 343.9 + 20 * 340.9 + disease * 332.1 + cancer * 319.9 + 1993 * 318.3 + games * 317.0 + years * 306.5 + patients * 299.8 + good * 286.3
Topic # 1
don * 1454.3 + just * 1392.8 + like * 1190.8 + know * 1178.1 + people * 836.9 + said * 802.5 + think * 799.7 + time * 754.2 + ve * 676.3 + didn * 675.9 + right * 636.3 + going * 625.4 + say * 620.7 + ll * 583.9 + way * 570.3
Topic # 2
image * 1047.7 + file * 999.1 + jpeg * 799.1 + program * 495.6 + gif * 466.0 + images * 443.7 + output * 442.3 + format * 442.3 + files * 438.5 + color * 406.3 + entry * 387.6 + 00 * 334.8 + use * 308.5 + bit * 308.4 + 03 * 258.7
Topic # 3
like * 620.7 + know * 591.7 + don * 543.7 + think * 528.4 + use * 514.3 + does * 510.2 + just * 509.1 + good * 425.8 + time * 417.4 + book * 410.7 + read * 402.9 + information * 395.2 + people * 393.5 + used * 388.2 + post * 368.4
Topic # 4
armenian * 960.6 + israel * 815

In [ ]:
lda.components_

### 개별 문서별 토픽 분포 확인
lda 객체의 transform()을 수행하면 개별 문서별 토픽 분포를 반환함

In [22]:
doc_topics = lda.transform(feat_vect)
print(doc_topics.shape)
print(doc_topics[:3])

# 7862 : document count , 8 : topic term(단어) count

(7862, 8)
[[0.01389701 0.01394362 0.01389104 0.48221844 0.01397882 0.01389205
  0.01393501 0.43424401]
 [0.27750436 0.18151826 0.0021208  0.53037189 0.00212129 0.00212102
  0.00212113 0.00212125]
 [0.00544459 0.22166575 0.00544539 0.00544528 0.00544039 0.00544168
  0.00544182 0.74567512]]


### 개별 문서별 토픽 분포도를 출력
- 20newsgroup으로 만들어진 문서명을 출력
- fetch_20newsgroup()으로 만들어진 데이터의 filename속성은 모든 문서의 문서명을 가지고 있음
- filename 속성은 절대 디렉토리를 가지는 문서명을 가지고 있으므로 '\'로 분할하여 맨 마지작 두번째 부터 파일명을 가져옴

In [28]:
def get_filename_list(newsdata):
    filename_list = []
    
    for file in newsdata.filenames:
#         print(file)
        filename_temp = file.split('\\')[-2:]
        filename = '.'.join(filename_temp)
        filename_list.append(filename)
    return filename_list

In [29]:
filename_list = get_filename_list(news_df)

print('filename 개수 : ', len(filename_list), ' filename list 10개만 : ', filename_list[:10])

filename 개수 :  7862  filename list 10개만 :  ['soc.religion.christian.20630', 'sci.med.59422', 'comp.graphics.38765', 'comp.graphics.38810', 'sci.med.59449', 'comp.graphics.38461', 'comp.windows.x.66959', 'rec.motorcycles.104487', 'sci.electronics.53875', 'sci.electronics.53617']


In [30]:
import pandas as pd

topic_names = ['Topic #' + str(i) for i in range(0,8)]

doc_topic_df = pd.DataFrame(
    data=doc_topics   # transform 한 애
    , columns=topic_names
    , index=filename_list
)

doc_topic_df.head()

,Topic #0,Topic #1,Topic #2,Topic #3,Topic #4,Topic #5,Topic #6,Topic #7
soc.religion.christian.20630,0.013897,0.013944,0.013891,0.482218,0.013979,0.013892,0.013935,0.434244
sci.med.59422,0.277504,0.181518,0.002121,0.530372,0.002121,0.002121,0.002121,0.002121
comp.graphics.38765,0.005445,0.221666,0.005445,0.005445,0.005440,0.005442,0.005442,0.745675
comp.graphics.38810,0.005439,0.005441,0.005449,0.578959,0.005440,0.388387,0.005442,0.005442
sci.med.59449,0.006584,0.552000,0.006587,0.408485,0.006585,0.006585,0.006588,0.006585
